<a href="https://colab.research.google.com/github/mattiapocci/PhilosoperRank/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Imports and Drive Mount***

In [62]:

#!pip3 install -U spacy[cuda92]
#!python3 -m spacy download en_core_web_lg
#https://stackoverflow.com/questions/54334304/spacy-cant-find-model-en-core-web-sm-on-windows-10-and-python-3-5-3-anacon

import sys
import time
import os
import numpy as np
import random
import json
import spacy
from spacy import displacy
import nltk
import pprint

nltk.download('punkt')

nlp = spacy.load("en_core_web_sm")
spacy.prefer_gpu()

from google.colab import drive

drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#***Import JSON articles***


In [0]:
"""
{
    "philosopher": "name",
    "article": "plaintext_article",
    "pageid": "id",
    "table_influenced": 
        [
            {
                "0": "name_of_someone_philosopher_influenced_by"
            }
        ]
    "table_influences": {
        [
            {
                "0": "name_of_someone_philosopher_influences"
            }
        ]
    }
}
"""
#open the file
with open('/content/drive/My Drive/folder/result_1.json') as f:
  jsonlist = json.load(f)

# ***Language Analysis***

In [0]:

"""
1. X cite(s|d) Y1, Y2, …, Yn as (an) influence(s)
2. X was influenced by Y1, Y2, …, Yn
3. Y has been cited as an influence by X1, X2, …, Xn
4. Y influence on X1, X2, …, Xn …
5. Y1, Y2, …, Yn influenced hi(m|s) …
"""

doc1 = nlp("Mark was influenced by Jhon")
doc2 = nlp("Jhon influenced Mark")
doc3 = nlp("Mark was an influence of Jhon and Albert")
doc4 = nlp("Jhon influenced him")
doc5 = nlp("Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts ")
doc6 = nlp("He is cited as an influence on John Milton")
print("------")
print("1 Mark was influenced by Jhon")
displacy.render(doc1, style="dep", jupyter=True)
print("CHUNKS")
for chunk in doc1.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc1:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("------")
print("2 Jhon influenced Mark")
displacy.render(doc2, style="dep", jupyter=True)
print("CHUNKS")

for chunk in doc2.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc2:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("------")
print("3 Mark was an influence of Jhon and Albert")
displacy.render(doc3, style="dep", jupyter=True)
print("CHUNKS")
    
for chunk in doc3.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc3:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("------")
print("4 Jhon influenced him")
displacy.render(doc4, style="dep", jupyter=True)
print("CHUNKS")
    
for chunk in doc4.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc4:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

print("5 Jhon influenced him")
displacy.render(doc5, style="dep", jupyter=True)
print("CHUNKS")
    
for chunk in doc5.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc5:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("6 Jhon influenced him")
displacy.render(doc5, style="dep", jupyter=True)
print("CHUNKS")
    
for chunk in doc5.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc5:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

    
#print(jsonlist[0]["article"])
#doc = nlp(jsonlist[0]["article"])

# ***Text Preprocessing***

In [0]:
"""
#HIM
#divide in sentences the article

#eliminate sentences that don't include influenc -e, -ed, -es, etc

#pattern matching on patterns (see language analysis)

#Entity recognition and recostruction

#US
"mark influenced jhon. Today is friday"
#divide in sentences the article
"mark influenced jhon"
"Today is friday"
#eliminate sentences that don't include influenc -e, -ed, -es, etc
"mark influenced jhon"

#pattern matching on patterns (using spacy 
?)
[x = "mark", y = "jhon"]

#Entity recognition and recostruction(here we have a proble, when he had Hendrix he could )
"""
from nltk.tokenize import word_tokenize
for elem in jsonlist:
    #divide in sentences the article
    sent_list = nltk.sent_tokenize(elem["article"])


    
    sent_list = [word_tokenize(i) for i in sent_list]

    influence_list = []
    influence_declinations = ["influence", "influenced", "influences", "influencing"]

    for word_list in sent_list:
        temp = [x for x in word_list if x in influence_declinations]
        if len(temp) != 0:
            influence_list.append(' '.join(word for word in word_list))

    new_list = []
    for sent in influence_list:
        new_list.append(sent)
        #displacy.render(sent, style="dep", jupyter=True)
    elem["article"] = new_list

# ***Processing***

In [72]:
print(jsonlist)

[{'pageid': '13692155', 'philosopher': 'Philosophy', 'article': ["The history of philosophy ( study of a specific period , individual or school ) is related to but not the same as the philosophy of history ( the theoretical aspect of history , which deals with questions such as the nature of historical evidence and the possibility of objectivity ) .Hegel 's Lectures on the Philosophy of History influenced many philosophers to interpret truth in light of history , a view called historicism ."]}, {'pageid': '691810', 'philosopher': 'Category:Philosophy', 'article': []}, {'pageid': '3072809', 'philosopher': 'Glossary of philosophy', 'article': ['Secularism , or religious freedom , is usually considered to go both ways : the state should not compel the people to follow ( or not follow ) a religion , and likewise religious doctrines should not influence the actions of the state .']}, {'pageid': '3073531', 'philosopher': 'Outline of philosophy', 'article': ['It exerts influence on a large ra